<a href="https://colab.research.google.com/github/finardi/Ranking/blob/main/1_Cobert_Tokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -q transformers

In [ ]:
import gc
import os
import torch
import pickle
import numpy as np
import pandas as pd
from functools import partial
from transformers import BertPreTrainedModel, BertModel, BertTokenizerFast


# better pandas viz
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', 100)
pd.set_option('max_colwidth', 700)
pd.set_option('display.max_rows', 5000)
  
# save/load pickles
def pickle_file(path, data=None):
    if data is None:
        with open(path, 'rb') as f:
            return pickle.load(f)
    if data is not None:
        with open(path, 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
 
# path base
path_base =  '/content/drive/MyDrive/ColBERT/ColBERT - FAQ Receita Federal/'

# load dataframes
df = pd.read_parquet(path_base+'data/df_FAQ_TRAIN.parquet.gzip')

print(f'unique docs:      {df["doc"].nunique()}')
print(f'unique questions: {df["query"].nunique()}')
df_triplet = pd.read_parquet(path_base+'data/df_FAQ_triplet_IDS_TRAIN.parquet.gzip')

# load data dicts
query_to_qid = pickle_file(path_base+'data/query_to_qid_TRAIN' )
qid_to_query = pickle_file(path_base+'data/qid_to_query_TRAIN')
doc_to_pid   = pickle_file(path_base+'data/doc_to_pid_TRAIN' )
pid_to_doc   = pickle_file(path_base+'data/pid_to_doc_TRAIN')

df.head()

unique docs:      612
unique questions: 612


,qid,query,pid,doc
0,0,pessoa física desobrigada pode apresentar a declaração de ajuste anual ?,0,"sim. pessoa física, ainda que desobrigada, pode apresentar a declaração de ajuste anual , sendo vedado a um mesmo contribuinte constar simultaneamente em mais de uma declaração de ajuste anual, seja como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de 2019."
1,1,qual é a forma de apuração do resultado da atividade rural da pessoa física?,1,"resultado da exploração da atividade rural exercida pela pessoa física é apurado mediante a escrituração do livro-caixa, abrangendo as receitas, as despesas, os investimentos e demais valores que integram a atividade. escrituração e a apuração devem ser feitas separadamente, por contribuinte e por país, em relação a todas as unidades rurais exploradas individualmente, em conjunto ou em comunhão em decorrência do regime de casamento. quando a receita bruta total auferida no ano-calendário não exceder a $ 56.000,00, é permitida a apuração mediante prova documental, dispensada a escrituração do livro-caixa, encontrando-se o resultado pela diferença entre o total das receitas e o..."
2,2,"contribuinte que constou como responsável perante a secretaria especial da receita federal do brasil por cadastro nacional da pessoa jurídica de associações (bairros, creches, clubes etc.) no ano-calendário de 2019, deve apresentar a declaração de ajuste anual do exercício de 2020?",2,"esse contribuinte está obrigado a declarar caso se enquadre nas hipóteses previstas na pergunta 001. não é o fato de ter constado como responsável perante a secretaria especial da receita federal do brasil por cadastro nacional da pessoa jurídica de associações (bairros, creches, clubes etc.), por si só, que obriga a apresentação de declaração de ajuste anual."
3,3,qual é o tratamento tributário dos rendimentos pagos ao sócio de serviço a título de pro- labore?,3,"incide imposto sobre a renda, na fonte e na declaração de ajuste anual, sobre os valores pagos ao sócio de serviço (consulte a solução de consulta interna cosit nº 12, de 15 de maio de 2013, disponível no sítio da na internet: http://receita.economia.gov.br => legislação => soluções de consultas e de divergências => sistema padrão de pesquisas da legislação da receita federal), a título de pro labore (rendimentos de trabalho). no entanto, não incide imposto sobre a renda sobre valores pagos a título de distribuição de lucros pelas pessoas jurídicas."
4,4,"animais, produtos ou bens rurais entregues para integralizar quotas subscritas em sociedade (empresa rural) configuram receita da atividade rural?",4,"sim, a entrega de animais, produtos ou bens rurais para integralização de capital em sociedade por quotas implica obtenção de receita e, em consequência, deve compor o resultado da atividade rural. valor pelo qual os animais, produtos ou bens rurais forem transferidos deve ser incluído como receita para apuração do rendimento tributável."


# Explore 1 row from df_triplet

In [ ]:
# triplet format
df_triplet.head()

,qid,pos_pid,neg_pid
0,0,0,582
1,1,1,509
2,2,2,384
3,3,3,84
4,4,4,311


In [ ]:
# positive doc to qid 0
pid_to_doc[0]

'sim.  pessoa física, ainda que desobrigada, pode apresentar a declaração de ajuste anual  , sendo  vedado a um mesmo contribuinte constar simultaneamente em mais de uma declaração de ajuste anual, seja  como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de  2019.'

In [ ]:
# negaitive doc to qid 0
pid_to_doc[248]

'contribuinte que receber rendimentos do trabalho não assalariado, inclusive os titulares de serviços notariais  e de registro e os leiloeiros podem deduzir, da receita decorrente do exercício da respectiva atividade, as  seguintes despesas escrituradas em livro-caixa:   1 - a remuneração paga a terceiros, desde que com vínculo empregatício, e os respectivos encargos  trabalhistas e previdenciários;   2 - os emolumentos pagos a terceiros, assim considerados os valores referentes à retribuição pela execução,  pelos serventuários públicos, de atos cartorários, judiciais e extrajudiciais;   3 - as despesas de custeio pagas, necessárias à percepção da receita e a manutenção da fonte produtora;      170   4 - as importâncias pagas, devidas aos empregados em decorrência das relações de trabalho, ainda que não  integrem a remuneração destes, caso configurem despesas necessárias à percepção da receita e à  manutenção da fonte produtora, observando-se que na hipótese de convenções e acordos col

# CONSTANTS

In [ ]:
bsize        = 3 # N
query_maxlen = 36
doc_maxlen   = 64

# Query Tokenization

In [ ]:
# =============================================
# ✨ step 0: build one batch with quries texts
# =============================================
query_bsize = df_triplet.qid.iloc[:bsize].to_list()
print(query_bsize)
query_bsize = [
            qid_to_query[query_bsize[0]], 
            qid_to_query[query_bsize[1]],
            qid_to_query[query_bsize[2]], 
              ]
query_bsize              

[0, 1, 2]


['pessoa física desobrigada pode apresentar a declaração de ajuste anual  ?',
 'qual é a forma de apuração do resultado da atividade rural da pessoa física?',
 'contribuinte que constou como responsável perante a secretaria especial da receita federal  do brasil   por cadastro nacional da pessoa jurídica   de associações (bairros, creches,  clubes etc.) no ano-calendário de 2019, deve apresentar a declaração de ajuste anual do exercício  de 2020?']

In [ ]:
# ===============================
# ✨ step 1: init BERT tokenizer
# ===============================
tok = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

In [ ]:
# ===========================
# ✨ step 2: build [Q] token
# ===========================
Q_marker_token, Q_marker_token_id = '[Q]', tok.convert_tokens_to_ids('[unused0]')
Q_marker_token, Q_marker_token_id

('[Q]', 100)

In [ ]:
# =================================================================
# ✨ step 3: add a place holder for token [Q] in the query's batch
# =================================================================
q_batch_text = ['. ' + x for x in query_bsize]
q_batch_text

['. pessoa física desobrigada pode apresentar a declaração de ajuste anual  ?',
 '. qual é a forma de apuração do resultado da atividade rural da pessoa física?',
 '. contribuinte que constou como responsável perante a secretaria especial da receita federal  do brasil   por cadastro nacional da pessoa jurídica   de associações (bairros, creches,  clubes etc.) no ano-calendário de 2019, deve apresentar a declaração de ajuste anual do exercício  de 2020?']

In [ ]:
# =====================================
# ✨ step 4: build the q_obj tokenizer
# =====================================
q_obj = tok(
    q_batch_text, 
    padding='max_length', 
    truncation=True,
    return_tensors='pt', 
    max_length=query_maxlen,
    )

# use only ids e mask keys in the q_obj dict
q_ids, q_mask = q_obj['input_ids'], q_obj['attention_mask']
q_ids, q_mask

(tensor([[  101,   119, 37344, 20291, 10143, 26218, 48855, 10250, 14396, 43787,
          10131,   143, 58262, 11115, 10102, 13657, 34165, 26474,   136,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0],
         [  101,   119, 13249,   147,   143, 11098, 10102, 15768, 11846, 11115,
          10154, 20318, 10141, 64313, 14982, 10141, 37344, 20291,   136,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0],
         [  101,   119, 52529, 76665, 11223, 10126, 10173, 15160, 10136, 10245,
          48061, 55852, 10218,   143, 34513, 16975, 10141, 58593, 11255, 12501,
          10154, 12369, 10190, 11822, 31559, 11275, 10141, 37344, 73370, 10102,
          13967, 52078, 12965,   113, 36175,   102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# =============================================================================
# ✨ step 5: insert [Q] after [CLS] token and switch the token [PAD] by [MASK]
# =============================================================================
q_ids[:, 1] = Q_marker_token_id
q_ids[q_ids == 0] = tok.mask_token_id

q_ids, q_mask

(tensor([[  101,   100, 37344, 20291, 10143, 26218, 48855, 10250, 14396, 43787,
          10131,   143, 58262, 11115, 10102, 13657, 34165, 26474,   136,   102,
            103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
            103,   103,   103,   103,   103,   103],
         [  101,   100, 13249,   147,   143, 11098, 10102, 15768, 11846, 11115,
          10154, 20318, 10141, 64313, 14982, 10141, 37344, 20291,   136,   102,
            103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
            103,   103,   103,   103,   103,   103],
         [  101,   100, 52529, 76665, 11223, 10126, 10173, 15160, 10136, 10245,
          48061, 55852, 10218,   143, 34513, 16975, 10141, 58593, 11255, 12501,
          10154, 12369, 10190, 11822, 31559, 11275, 10141, 37344, 73370, 10102,
          13967, 52078, 12965,   113, 36175,   102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# =====================================================
# ✨ step 6: build the query's batch: list(ids, mask)
# =====================================================
q_batches = []
# range (0, N, N)
for offset in range(0, q_ids.size(0), bsize):
    q_batches.append((q_ids[offset:offset+bsize], q_mask[offset:offset+bsize]))
q_batches    

[(tensor([[  101,   100, 37344, 20291, 10143, 26218, 48855, 10250, 14396, 43787,
           10131,   143, 58262, 11115, 10102, 13657, 34165, 26474,   136,   102,
             103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
             103,   103,   103,   103,   103,   103],
          [  101,   100, 13249,   147,   143, 11098, 10102, 15768, 11846, 11115,
           10154, 20318, 10141, 64313, 14982, 10141, 37344, 20291,   136,   102,
             103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
             103,   103,   103,   103,   103,   103],
          [  101,   100, 52529, 76665, 11223, 10126, 10173, 15160, 10136, 10245,
           48061, 55852, 10218,   143, 34513, 16975, 10141, 58593, 11255, 12501,
           10154, 12369, 10190, 11822, 31559, 11275, 10141, 37344, 73370, 10102,
           13967, 52078, 12965,   113, 36175,   102]]),
  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
           0, 0, 0, 0, 0

# Doc Tokenization

In [ ]:
df_triplet.iloc[:bsize]

,qid,pos_pid,neg_pid
0,0,0,582
1,1,1,509
2,2,2,384


In [ ]:
# ===========================================================
# ✨ step 0: build one batch with positive and negative docs 
# ===========================================================
doc_bsize = df.doc.iloc[:bsize].to_list()

doc_bsize_pos = df_triplet.pos_pid.iloc[:bsize].to_list()
doc_bsize_neg = df_triplet.neg_pid.iloc[:bsize].to_list()
print('pos_pids', doc_bsize_pos)
print('neg_pids', doc_bsize_neg)
pos_docs = [
            pid_to_doc[doc_bsize_pos[0]], 
            pid_to_doc[doc_bsize_pos[1]],
            pid_to_doc[doc_bsize_pos[2]], 
           ]

neg_docs = [
            pid_to_doc[doc_bsize_neg[0]], 
            pid_to_doc[doc_bsize_neg[1]],
            pid_to_doc[doc_bsize_neg[2]], 
           ]

doc_bsize = pos_docs + neg_docs

pos_docs, neg_docs              

pos_pids [0, 1, 2]
neg_pids [582, 509, 384]


(['sim.  pessoa física, ainda que desobrigada, pode apresentar a declaração de ajuste anual  , sendo  vedado a um mesmo contribuinte constar simultaneamente em mais de uma declaração de ajuste anual, seja  como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de  2019.',
  'resultado da exploração da atividade rural exercida pela pessoa física é apurado mediante a escrituração  do livro-caixa, abrangendo as receitas, as despesas, os investimentos e demais valores que integram a  atividade.     escrituração e a apuração devem ser feitas separadamente, por contribuinte e por país, em relação a todas  as unidades rurais exploradas individualmente, em conjunto ou em comunhão em decorrência do regime de  casamento.   quando a receita bruta total auferida no ano-calendário não exceder a $ 56.000,00, é permitida a apuração  mediante prova documental, dispensada a escrituração do livro-caixa, encontrando-se o resultado pela  diferença entre o tot

In [ ]:
# ===============================
# ✨ step 1: build the [D] token
# ===============================
D_marker_token, D_marker_token_id = '[D]', tok.convert_tokens_to_ids('[unused1]')

# =================================================================
# ✨ step 2: add a place holder for token [Q] in the query's batch
# =================================================================
d_batch_text = ['. ' + x for x in doc_bsize]

# =====================================
# ✨ step 3: build the q_obj tokenizer
# =====================================
d_obj = tok(
    d_batch_text, 
    padding='max_length', 
    truncation=True,
    return_tensors='pt', 
    max_length=doc_maxlen,
    )

# utiliza somente ids e mask keys
d_ids, d_mask = d_obj['input_ids'], d_obj['attention_mask']

# =============================================================================
# ✨ step 4: insert [D] after [CLS] token and switch the token [PAD] by [MASK]
# =============================================================================
d_ids[:, 1] = D_marker_token_id

# ==================================
# ✨ step 5: sort the doc's lengths 
# ==================================
indices = d_mask.sum(-1).sort().indices
reverse_indices = indices.sort().indices
d_ids = d_ids[indices]
d_mask = d_mask[indices]

# ===================================================================
# ✨ step 7: build the doc's batch: list(ids, mask), reverse_indices
# ===================================================================
d_batches = []
for offset in range(0, d_ids.size(0), bsize):
    d_batches.append((d_ids[offset:offset+bsize], d_mask[offset:offset+bsize]))
d_batches    

[(tensor([[  101,     1, 12537, 10819, 53311,   147, 84177, 11977, 10120, 86852,
           52054, 10405,   157, 17859, 10102, 51801, 10154, 30106, 11259,   147,
             157, 10674, 93011, 10102, 23955, 45426, 12574,   119,   102,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0],
          [  101,     1, 33873,   119, 37344, 20291,   117, 13991, 10126, 10143,
           26218, 48855, 10250,   117, 14396, 43787, 10131,   143, 58262, 11115,
           10102, 13657, 34165, 26474,   117, 14297, 59441, 10351,   143, 10316,
           13688, 52529, 76665, 11223, 30736, 10131, 73690, 10252, 10557, 10102,
           10477, 58262, 11115, 10102, 13657, 34165, 26474,   117, 26625, 10245,
           25325, 10391, 38788, 10111,   117, 11460, 87088, 11499, 19

# Tensorize Triplets

In [ ]:
# =================================================
# ✨ step 0: assign  ids e mask tokens and reshape
# =================================================
N = len(query_bsize)
Q_ids, Q_mask = q_ids, q_mask
D_ids, D_mask = d_ids, d_mask
D_ids, D_mask = D_ids.view(2, N, -1), D_mask.view(2, N, -1)

print(f'Q_ids:  {q_ids.shape}')
print(f'Q_mask: {q_mask.shape}\n')

print(f'D_ids:                {d_ids.shape}')
print(f'D_ids.view(2, N, -1): {d_ids.view(2, N, -1).shape}\n')
print(f'D_mask:                {d_mask.shape}')
print(f'D_mask.view(2, N, -1): {d_mask.view(2, N, -1).shape}')

Q_ids:  torch.Size([3, 36])
Q_mask: torch.Size([3, 36])

D_ids:                torch.Size([6, 64])
D_ids.view(2, N, -1): torch.Size([2, 3, 64])

D_mask:                torch.Size([6, 64])
D_mask.view(2, N, -1): torch.Size([2, 3, 64])


In [ ]:
# ==========================================================================================================
# ✨ step 1: get the max value between the len of i-th positive and the len of the i-th negative for i in N
# ==========================================================================================================
maxlens = D_mask.sum(-1).max(0).values
print(maxlens)
indices = maxlens.sort().indices
indices

tensor([64, 64, 64])


tensor([0, 1, 2])

In [ ]:
# =====================================
# ✨ step 2: sort Q_* e D_* by maxlens
# =====================================
indices = maxlens.sort().indices
Q_ids, Q_mask = Q_ids[indices], Q_mask[indices]
D_ids, D_mask = D_ids[:, indices], D_mask[:, indices]

In [ ]:
# ==========================================================================
# ✨ step 3: split the positive e negative ids and mask from D_ids e D_mask
# ==========================================================================
(positive_ids, negative_ids), (positive_mask, negative_mask) = D_ids, D_mask
positive_ids.shape, negative_ids.shape, positive_mask.shape, negative_mask.shape

(torch.Size([3, 64]),
 torch.Size([3, 64]),
 torch.Size([3, 64]),
 torch.Size([3, 64]))

In [ ]:
# =====================================================================
# ✨ step 4: build batches to queries, positive_docs and negative_docs
# =====================================================================
(positive_ids, negative_ids), (positive_mask, negative_mask) = D_ids, D_mask

query_batches = []
for offset in range(0, Q_ids.size(0), bsize):
    query_batches.append((Q_ids[offset:offset+bsize], Q_mask[offset:offset+bsize]))

positive_batches = []
for offset in range(0, positive_ids.size(0), bsize):
    positive_batches.append((positive_ids[offset:offset+bsize], positive_mask[offset:offset+bsize]))    

negative_batches = []
for offset in range(0, negative_ids.size(0), bsize):
    negative_batches.append((negative_ids[offset:offset+bsize], negative_mask[offset:offset+bsize]))    

In [ ]:
# ===================================================================
# ✨ step 5: group the batches: (query_, positive_, negative)batches
# ===================================================================
batches = []
for (q_ids, q_mask), (p_ids, p_mask), (n_ids, n_mask) in zip(query_batches, positive_batches, negative_batches):
    Q = (torch.cat((q_ids, q_ids)), torch.cat((q_mask, q_mask))) # <- duplicate Q (one to pos docs and another for neg docs)
    D = (torch.cat((p_ids, n_ids)), torch.cat((p_mask, n_mask)))
    batches.append((Q, D))
batches

[((tensor([[  101,   100, 37344, 20291, 10143, 26218, 48855, 10250, 14396, 43787,
            10131,   143, 58262, 11115, 10102, 13657, 34165, 26474,   136,   102,
              103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
              103,   103,   103,   103,   103,   103],
           [  101,   100, 13249,   147,   143, 11098, 10102, 15768, 11846, 11115,
            10154, 20318, 10141, 64313, 14982, 10141, 37344, 20291,   136,   102,
              103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
              103,   103,   103,   103,   103,   103],
           [  101,   100, 52529, 76665, 11223, 10126, 10173, 15160, 10136, 10245,
            48061, 55852, 10218,   143, 34513, 16975, 10141, 58593, 11255, 12501,
            10154, 12369, 10190, 11822, 31559, 11275, 10141, 37344, 73370, 10102,
            13967, 52078, 12965,   113, 36175,   102],
           [  101,   100, 37344, 20291, 10143, 26218, 48855, 10250, 14396, 43787,
            101

# In class format

In [ ]:
# =============
# ✨ Constants
# =============
bsize = 16 # N
query_maxlen = 48
doc_maxlen = 128
path_model = 'bert-base-multilingual-uncased'

# ==================
# ✨ QueryTokenizer
# ==================
class QueryTokenizer():
    def __init__(self, query_maxlen, path_tokenizer):
        self.tok = BertTokenizerFast.from_pretrained(path_tokenizer)
        self.query_maxlen = query_maxlen

        self.cls_token, self.cls_token_id = self.tok.cls_token, self.tok.cls_token_id
        self.sep_token, self.sep_token_id = self.tok.sep_token, self.tok.sep_token_id
        self.mask_token, self.mask_token_id = self.tok.mask_token, self.tok.mask_token_id

    def tokenize(self, batch_text, add_special_tokens=False):
        assert type(batch_text) in [list, tuple], (type(batch_text))

        tokens = [self.tok.tokenize(x, add_special_tokens=False) for x in batch_text]

        if not add_special_tokens:
            return tokens

        prefix, suffix = [self.cls_token], [self.sep_token]
        tokens = [prefix + lst + suffix + [self.mask_token] * (self.query_maxlen - (len(lst)+3)) for lst in tokens]

        return tokens

    def encode(self, batch_text, add_special_tokens=False):
        assert type(batch_text) in [list, tuple], (type(batch_text))

        ids = self.tok(batch_text, add_special_tokens=False)['input_ids']

        if not add_special_tokens:
            return ids

        prefix, suffix = [self.cls_token_id], [self.sep_token_id]
        ids = [prefix + lst + suffix + [self.mask_token_id] * (self.query_maxlen - (len(lst)+3)) for lst in ids]

        return ids

    def tensorize(self, batch_text, bsize=None):
        assert type(batch_text) in [list, tuple], (type(batch_text))

        obj = self.tok(batch_text, padding='max_length', truncation=True,
                       return_tensors='pt', max_length=self.query_maxlen)

        ids, mask = obj['input_ids'], obj['attention_mask']

        ids[ids == 0] = self.mask_token_id

        if bsize:
            batches = _split_into_batches(ids, mask, bsize)
            return batches

        return ids, mask

# ================
# ✨ DocTokenizer
# ================
class DocTokenizer():
    def __init__(self, doc_maxlen, path_tokenizer):
        self.tok = BertTokenizerFast.from_pretrained(path_tokenizer)
        self.doc_maxlen = doc_maxlen

        self.cls_token, self.cls_token_id = self.tok.cls_token, self.tok.cls_token_id
        self.sep_token, self.sep_token_id = self.tok.sep_token, self.tok.sep_token_id

    def tokenize(self, batch_text, add_special_tokens=False):
        assert type(batch_text) in [list, tuple], (type(batch_text))

        tokens = [self.tok.tokenize(x, add_special_tokens=False) for x in batch_text]

        if not add_special_tokens:
            return tokens

        prefix, suffix = [self.cls_token], [self.sep_token]
        tokens = [prefix + lst + suffix for lst in tokens]

        return tokens

    def encode(self, batch_text, add_special_tokens=False):
        assert type(batch_text) in [list, tuple], (type(batch_text))

        ids = self.tok(batch_text, add_special_tokens=False)['input_ids']

        if not add_special_tokens:
            return ids

        prefix, suffix = [self.cls_token_id], [self.sep_token_id]
        ids = [prefix + lst + suffix for lst in ids]

        return ids

    def tensorize(self, batch_text, bsize=None):
        assert type(batch_text) in [list, tuple], (type(batch_text))

        obj = self.tok(batch_text, padding='longest', truncation='longest_first',
                       return_tensors='pt', max_length=self.doc_maxlen)

        ids, mask = obj['input_ids'], obj['attention_mask']

        if bsize:
            ids, mask, reverse_indices = _sort_by_length(ids, mask, bsize)
            batches = _split_into_batches(ids, mask, bsize)
            return batches, reverse_indices

        return ids, mask

# =====================
# ✨ tensorize triples
# =====================
def tensorize_triples(query_tokenizer, doc_tokenizer, queries, positives, negatives, bsize):
    assert len(queries) == len(positives) == len(negatives)
    assert bsize is None or len(queries) % bsize == 0

    N = len(queries)
    assert bsize == N
    Q_ids, Q_mask = query_tokenizer.tensorize(queries)
    D_ids, D_mask = doc_tokenizer.tensorize(positives + negatives)
    D_ids, D_mask = D_ids.view(2, N, -1), D_mask.view(2, N, -1)

    # Compute max among {length of i^th positive, length of i^th negative} for i \in N
    maxlens = D_mask.sum(-1).max(0).values

    # Sort by maxlens
    indices = maxlens.sort().indices
    Q_ids, Q_mask = Q_ids[indices], Q_mask[indices]
    D_ids, D_mask = D_ids[:, indices], D_mask[:, indices]

    (positive_ids, negative_ids), (positive_mask, negative_mask) = D_ids, D_mask

    query_batches = _split_into_batches(Q_ids, Q_mask, bsize)
    positive_batches = _split_into_batches(positive_ids, positive_mask, bsize)
    negative_batches = _split_into_batches(negative_ids, negative_mask, bsize)

    batches = []
    for (q_ids, q_mask), (p_ids, p_mask), (n_ids, n_mask) in zip(query_batches, positive_batches, negative_batches):
        Q = (torch.cat((q_ids, q_ids)), torch.cat((q_mask, q_mask)))
        D = (torch.cat((p_ids, n_ids)), torch.cat((p_mask, n_mask)))
        batches.append((Q, D))

    return batches

# =============
# ✨ Aux funcs
# =============
def _sort_by_length(ids, mask, bsize):
    if ids.size(0) <= bsize:
        return ids, mask, torch.arange(ids.size(0))

    indices = mask.sum(-1).sort().indices
    reverse_indices = indices.sort().indices

    return ids[indices], mask[indices], reverse_indices

def _split_into_batches(ids, mask, bsize):
    batches = []
    for offset in range(0, ids.size(0), bsize):
        batches.append((ids[offset:offset+bsize], mask[offset:offset+bsize]))

    return batches

# ===============
# ✨ LazyBatcher
# ===============
class LazyBatcher():
    def __init__(self, bsize, path, path_tokenizer, query_maxlen, doc_maxlen, mode='train', accumsteps=1):
        self.bsize, self.accumsteps = bsize, accumsteps
        self.query_tokenizer = QueryTokenizer(query_maxlen=query_maxlen, path_tokenizer=path_tokenizer)
        self.doc_tokenizer = DocTokenizer(doc_maxlen=doc_maxlen, path_tokenizer=path_tokenizer)
        self.tensorize_triples = partial(tensorize_triples, self.query_tokenizer, self.doc_tokenizer)
        self.position = 0
        self.mode = mode

        self.triples = self._load_triples(path_base)
        self.queries = self._load_queries(path_base)
        self.collection = self._load_collection(path_base)
    
    def _load_triples(self, path):
        if self.mode == 'train':
            path = path+'data/df_FAQ_triplet_IDS_TRAIN.parquet.gzip'
        elif self.mode == 'valid':
            path = path+'data/df_FAQ_triplet_IDS_VALID.parquet.gzip'

        df_triplet = pd.read_parquet(path)
        triples = []
        for qid, pos_pid, neg_pid in zip(
            df_triplet.qid.values,
            df_triplet.pos_pid.values,
            df_triplet.neg_pid.values
            ):
            triples.append((qid, pos_pid, neg_pid))

        return triples

    def _load_queries(self, path):
        if self.mode == 'train':
            qid_to_query_train = path+'data/qid_to_query_TRAIN'
            return pickle_file(qid_to_query_train)
        elif self.mode == 'valid':
            qid_to_query_valid = path+'data/qid_to_query_VALID'
            return pickle_file(qid_to_query_valid)

    def _load_collection(self, path):
        if self.mode == 'train':
            pid_to_doc_train = path+'data/pid_to_doc_TRAIN'
            return pickle_file(pid_to_doc_train)
        elif self.mode == 'valid':
            pid_to_doc_valid = path+'data/pid_to_doc_VALID'
            return pickle_file(pid_to_doc_valid)
        

    def __iter__(self):
        return self

    def __len__(self):
        return len(self.triples)

    def __next__(self):
        # offsets determines the starting index position of each bag (sequence) in input.
        offset, endpos = self.position, min(self.position + self.bsize, len(self.triples))
        self.position = endpos

        if offset + self.bsize > len(self.triples):
            raise StopIteration

        queries, positives, negatives = [], [], []

        for position in range(offset, endpos):
            query, pos, neg = self.triples[position]
            query, pos, neg = self.queries[query], self.collection[pos], self.collection[neg]
            queries.append(query)
            positives.append(pos)
            negatives.append(neg)

        return self.collate(queries, positives, negatives)

    def collate(self, queries, positives, negatives):
        assert len(queries) == len(positives) == len(negatives) == self.bsize

        return self.tensorize_triples(queries, positives, negatives, self.bsize // self.accumsteps)

# - - - - -
dataloader_train = LazyBatcher(
    bsize=bsize, 
    path=path_base, 
    path_tokenizer=path_model,
    query_maxlen=query_maxlen,
    doc_maxlen=doc_maxlen,
    mode='train'
    )
print('batches:')
for i, batches in enumerate(dataloader_train):
    print(f' {i }.', end ='')

batch:
 0. 1. 2. 3. 4. 5. 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.

In [ ]:
# ===================
# ✨ Print one batch
# ===================
dataloader = LazyBatcher(
    bsize=bsize, 
    path=path_base, 
    path_tokenizer=path_model,
    query_maxlen=query_maxlen,
    doc_maxlen=doc_maxlen,
    mode='valid'
    )

dl0 = next(iter(dataloader))
dl0

[((tensor([[  101, 10146, 10143,  ...,   103,   103,   103],
           [  101, 10245, 43787,  ...,   103,   103,   103],
           [  101, 13249,   147,  ...,   103,   103,   103],
           ...,
           [  101, 10245, 11132,  ...,   103,   103,   103],
           [  101, 23840, 11132,  ...,   103,   103,   103],
           [  101, 13249,   147,  ...,   103,   103,   103]]),
   tensor([[1, 1, 1,  ..., 0, 0, 0],
           [1, 1, 1,  ..., 0, 0, 0],
           [1, 1, 1,  ..., 0, 0, 0],
           ...,
           [1, 1, 1,  ..., 0, 0, 0],
           [1, 1, 1,  ..., 0, 0, 0],
           [1, 1, 1,  ..., 0, 0, 0]])),
  (tensor([[  101, 11373,   119,  ...,     0,     0,     0],
           [  101, 52813, 22107,  ..., 12621, 26065,   102],
           [  101, 10128, 11373,  ...,     0,     0,     0],
           ...,
           [  101, 11373,   117,  ...,   113, 37310,   102],
           [  101, 17859, 19719,  ..., 89755, 71162,   102],
           [  101,   100, 36362,  ...,   143, 42761,  